https://medium.com/swlh/web-scraping-python-requests-and-beautifulsoup-45d5f48f5a1

https://sparkbyexamples.com/spark/spark-read-text-file-rdd-dataframe/

https://medium.com/swlh/text2sql-in-spark-nlp-converting-natural-language-questions-to-sql-queries-on-scale-6ae9a9061d74

https://www.oreilly.com/library/view/learning-spark-2nd/9781492050032/ch04.html

In [1]:
!python -m pip install html2text

In [2]:
!python -m pip install mistletoe

https://github.com/webrecorder/warcio

In [3]:
!pip install warcio

In [4]:
!python -m pip install cdx_toolkit

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import json
import sys
import os.path
import re
import logging
import pandas as pd
import requests
import warcio
import mistletoe
import cdx_toolkit
import zlib

from warcio import ArchiveIterator
from contextlib import closing
from html2text import HTML2Text
from bs4 import BeautifulSoup
from pathlib import Path
from IPython.display import HTML as HTML_raw, display

In [7]:
websites = pd.read_csv("./ccmain/ccmain_2021-04_Long2.txt", header = None, sep = '|')

In [8]:
websites.head()

,0,1,2,3
0,NaN,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
1,NaN,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
2,NaN,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
3,NaN,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...,NaN
4,NaN,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...,NaN


In [9]:
websites.columns

Int64Index([0, 1, 2, 3], dtype='int64')

In [10]:
websites.drop([0, 3], axis=1, inplace=True)
websites.head()

,1,2
0,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
1,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...
4,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...


In [11]:
websites.columns = ['url_link', 'warc_file']
websites.head()

,url_link,warc_file
0,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
1,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...
4,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...


In [12]:
websites.url_link.value_counts()

https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      7
https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe                                                               

In [13]:
websites.url_link.value_counts().keys()

Index(['https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ',
       'https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe                                                  

In [14]:
urlList = []
for i in websites.url_link.value_counts().keys():
    urlList.append(i.replace(' ',''))
    
print(urlList[:10])

['https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington', 'https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/151789/werkstudent-w-strich-m-strich-d-softwareentwicklung-java-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/152230/senior-software-engineer-w-strich-m-strich-d-sap-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.xn--itbaw-ova.de/jobs/152638/software-architect-w-strich-m-strich-d-java-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.xn--itbaw-ova.de/jobs/151969/senior-software-engineer-w-strich-m-strich-d-java-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.vdk.de/bayern/pages/presse/vdk-zeitung/vdk-zeitung_archiv/80394/minijob_sichert_anspruch', 'https://www.vdk.de/rheinland-pfalz/pages/80909/wenn_der_job_nicht_mehr_passt', 'https://help.workable.com/hc/en-us/articles/1150

In [15]:
statesList = ["United States of America","USA","AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS",
              "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA",
              "RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY","DC","GU","MH","MP","PR","VI"]
itList = ["engineer","Engineer","scientist","Scientist","developer","Developer","Analyst","analyst","Operator","operator"]
qaList = ["Qualification","Qualifications","qualification","qualifications","Certification","Certifications","certification",
          "certifications"]

In [16]:
fout = open("./ads/2021JAN_USAjobAD.txt","w")
for i in range(0, len(urlList)):
    filename = "./dat/"+str(i+1)+"_2021JAN_jobAD.txt"
    if (os.path.exists(filename)):
        with open(filename, encoding='utf-8') as fin:
            #all_USAs = re.findall('USA', fin.read())
            all_USAs = re.findall(r"(?=("+'|'.join(statesList)+r"))", fin.read())
            nUSAs = len(all_USAs)
            if nUSAs > 0:
                print(str(i+1)+"_2021JAN_jobAD.txt",' ',nUSAs)
                fout.write(str(i+1)+'_2021JAN_jobAD.txt'+' '+str(nUSAs)+'\n')
            else:
                continue
    else:
        continue
            
fout.close()

1_2021JAN_jobAD.txt   49
11_2021JAN_jobAD.txt   3
21_2021JAN_jobAD.txt   4
22_2021JAN_jobAD.txt   3
23_2021JAN_jobAD.txt   3
27_2021JAN_jobAD.txt   4
28_2021JAN_jobAD.txt   4
39_2021JAN_jobAD.txt   4
41_2021JAN_jobAD.txt   4
45_2021JAN_jobAD.txt   3
49_2021JAN_jobAD.txt   1
53_2021JAN_jobAD.txt   24
54_2021JAN_jobAD.txt   45
55_2021JAN_jobAD.txt   17
56_2021JAN_jobAD.txt   8
62_2021JAN_jobAD.txt   15
65_2021JAN_jobAD.txt   3
74_2021JAN_jobAD.txt   3
84_2021JAN_jobAD.txt   6
87_2021JAN_jobAD.txt   17
91_2021JAN_jobAD.txt   4
96_2021JAN_jobAD.txt   5
108_2021JAN_jobAD.txt   3
115_2021JAN_jobAD.txt   1
116_2021JAN_jobAD.txt   1
117_2021JAN_jobAD.txt   1
121_2021JAN_jobAD.txt   4
124_2021JAN_jobAD.txt   18
125_2021JAN_jobAD.txt   3
129_2021JAN_jobAD.txt   6
132_2021JAN_jobAD.txt   2
134_2021JAN_jobAD.txt   6
135_2021JAN_jobAD.txt   4
147_2021JAN_jobAD.txt   5
153_2021JAN_jobAD.txt   5
156_2021JAN_jobAD.txt   1
168_2021JAN_jobAD.txt   24
169_2021JAN_jobAD.txt   3
177_2021JAN_jobAD.txt   3
1

1074_2021JAN_jobAD.txt   4
1077_2021JAN_jobAD.txt   10
1083_2021JAN_jobAD.txt   4
1085_2021JAN_jobAD.txt   10
1086_2021JAN_jobAD.txt   14
1087_2021JAN_jobAD.txt   4
1091_2021JAN_jobAD.txt   6
1098_2021JAN_jobAD.txt   5
1099_2021JAN_jobAD.txt   6
1100_2021JAN_jobAD.txt   3
1102_2021JAN_jobAD.txt   4
1105_2021JAN_jobAD.txt   2
1106_2021JAN_jobAD.txt   11
1108_2021JAN_jobAD.txt   4
1110_2021JAN_jobAD.txt   8
1111_2021JAN_jobAD.txt   5
1117_2021JAN_jobAD.txt   5
1118_2021JAN_jobAD.txt   8
1120_2021JAN_jobAD.txt   118
1123_2021JAN_jobAD.txt   2
1124_2021JAN_jobAD.txt   5
1127_2021JAN_jobAD.txt   4
1135_2021JAN_jobAD.txt   200
1137_2021JAN_jobAD.txt   5
1139_2021JAN_jobAD.txt   11
1143_2021JAN_jobAD.txt   3
1144_2021JAN_jobAD.txt   22
1146_2021JAN_jobAD.txt   14
1147_2021JAN_jobAD.txt   620
1148_2021JAN_jobAD.txt   1
1149_2021JAN_jobAD.txt   187
1150_2021JAN_jobAD.txt   3
1152_2021JAN_jobAD.txt   52
1158_2021JAN_jobAD.txt   20
1160_2021JAN_jobAD.txt   14
1161_2021JAN_jobAD.txt   9
1163_2021J

2583_2021JAN_jobAD.txt   6
2585_2021JAN_jobAD.txt   10
2597_2021JAN_jobAD.txt   43
2602_2021JAN_jobAD.txt   5
2604_2021JAN_jobAD.txt   2
2605_2021JAN_jobAD.txt   11
2612_2021JAN_jobAD.txt   24
2620_2021JAN_jobAD.txt   14
2621_2021JAN_jobAD.txt   14
2623_2021JAN_jobAD.txt   11
2624_2021JAN_jobAD.txt   55
2635_2021JAN_jobAD.txt   10
2636_2021JAN_jobAD.txt   14
2639_2021JAN_jobAD.txt   20
2648_2021JAN_jobAD.txt   2
2650_2021JAN_jobAD.txt   14
2654_2021JAN_jobAD.txt   12
2658_2021JAN_jobAD.txt   22
2659_2021JAN_jobAD.txt   1
2660_2021JAN_jobAD.txt   62
2662_2021JAN_jobAD.txt   11
2665_2021JAN_jobAD.txt   11
2668_2021JAN_jobAD.txt   6
2671_2021JAN_jobAD.txt   11
2673_2021JAN_jobAD.txt   10
2675_2021JAN_jobAD.txt   10
2683_2021JAN_jobAD.txt   11
2684_2021JAN_jobAD.txt   4
2685_2021JAN_jobAD.txt   11
2687_2021JAN_jobAD.txt   18
2693_2021JAN_jobAD.txt   3
2696_2021JAN_jobAD.txt   2
2700_2021JAN_jobAD.txt   33
2701_2021JAN_jobAD.txt   3
2705_2021JAN_jobAD.txt   7
2706_2021JAN_jobAD.txt   15
270

3468_2021JAN_jobAD.txt   67
3469_2021JAN_jobAD.txt   3
3472_2021JAN_jobAD.txt   7
3474_2021JAN_jobAD.txt   15
3475_2021JAN_jobAD.txt   5
3476_2021JAN_jobAD.txt   4
3479_2021JAN_jobAD.txt   3
3480_2021JAN_jobAD.txt   7
3483_2021JAN_jobAD.txt   8
3494_2021JAN_jobAD.txt   13
3496_2021JAN_jobAD.txt   11
3497_2021JAN_jobAD.txt   11
3498_2021JAN_jobAD.txt   12
3505_2021JAN_jobAD.txt   15
3510_2021JAN_jobAD.txt   1
3514_2021JAN_jobAD.txt   11
3517_2021JAN_jobAD.txt   46
3521_2021JAN_jobAD.txt   177
3526_2021JAN_jobAD.txt   14
3527_2021JAN_jobAD.txt   4
3530_2021JAN_jobAD.txt   1
3533_2021JAN_jobAD.txt   3
3534_2021JAN_jobAD.txt   10
3535_2021JAN_jobAD.txt   6
3538_2021JAN_jobAD.txt   1
3540_2021JAN_jobAD.txt   10
3552_2021JAN_jobAD.txt   12
3554_2021JAN_jobAD.txt   9
3558_2021JAN_jobAD.txt   13
3564_2021JAN_jobAD.txt   32
3565_2021JAN_jobAD.txt   4
3568_2021JAN_jobAD.txt   2
3570_2021JAN_jobAD.txt   3
3572_2021JAN_jobAD.txt   4
3577_2021JAN_jobAD.txt   14
3580_2021JAN_jobAD.txt   34
3582_2021

4364_2021JAN_jobAD.txt   4
4366_2021JAN_jobAD.txt   13
4371_2021JAN_jobAD.txt   11
4376_2021JAN_jobAD.txt   10
4377_2021JAN_jobAD.txt   20
4378_2021JAN_jobAD.txt   4
4379_2021JAN_jobAD.txt   4
4382_2021JAN_jobAD.txt   2
4386_2021JAN_jobAD.txt   29
4393_2021JAN_jobAD.txt   11
4402_2021JAN_jobAD.txt   6
4404_2021JAN_jobAD.txt   1
4405_2021JAN_jobAD.txt   11
4407_2021JAN_jobAD.txt   9
4408_2021JAN_jobAD.txt   3
4410_2021JAN_jobAD.txt   14
4414_2021JAN_jobAD.txt   16
4415_2021JAN_jobAD.txt   3
4423_2021JAN_jobAD.txt   10
4425_2021JAN_jobAD.txt   11
4429_2021JAN_jobAD.txt   33
4430_2021JAN_jobAD.txt   2
4432_2021JAN_jobAD.txt   3
4435_2021JAN_jobAD.txt   3
4436_2021JAN_jobAD.txt   7
4437_2021JAN_jobAD.txt   14
4439_2021JAN_jobAD.txt   11
4445_2021JAN_jobAD.txt   4
4458_2021JAN_jobAD.txt   11
4459_2021JAN_jobAD.txt   83
4462_2021JAN_jobAD.txt   3
4463_2021JAN_jobAD.txt   11
4466_2021JAN_jobAD.txt   11
4468_2021JAN_jobAD.txt   11
4474_2021JAN_jobAD.txt   5
4477_2021JAN_jobAD.txt   11
4479_202

5381_2021JAN_jobAD.txt   11
5382_2021JAN_jobAD.txt   12
5392_2021JAN_jobAD.txt   2
5393_2021JAN_jobAD.txt   14
5397_2021JAN_jobAD.txt   9
5404_2021JAN_jobAD.txt   18
5408_2021JAN_jobAD.txt   4
5409_2021JAN_jobAD.txt   7
5411_2021JAN_jobAD.txt   3
5412_2021JAN_jobAD.txt   4
5413_2021JAN_jobAD.txt   3
5421_2021JAN_jobAD.txt   135
5423_2021JAN_jobAD.txt   8
5425_2021JAN_jobAD.txt   5
5431_2021JAN_jobAD.txt   1
5432_2021JAN_jobAD.txt   2
5442_2021JAN_jobAD.txt   13
5454_2021JAN_jobAD.txt   30
5456_2021JAN_jobAD.txt   1
5458_2021JAN_jobAD.txt   8
5459_2021JAN_jobAD.txt   177
5460_2021JAN_jobAD.txt   5
5466_2021JAN_jobAD.txt   3
5467_2021JAN_jobAD.txt   43
5469_2021JAN_jobAD.txt   20
5470_2021JAN_jobAD.txt   13
5471_2021JAN_jobAD.txt   12
5472_2021JAN_jobAD.txt   11
5473_2021JAN_jobAD.txt   7
5478_2021JAN_jobAD.txt   16
5482_2021JAN_jobAD.txt   2
5484_2021JAN_jobAD.txt   2
5490_2021JAN_jobAD.txt   129
5491_2021JAN_jobAD.txt   33
5492_2021JAN_jobAD.txt   8
5493_2021JAN_jobAD.txt   9
5498_2021

6492_2021JAN_jobAD.txt   17
6499_2021JAN_jobAD.txt   16
6509_2021JAN_jobAD.txt   1
6513_2021JAN_jobAD.txt   7
6514_2021JAN_jobAD.txt   8
6517_2021JAN_jobAD.txt   42
6518_2021JAN_jobAD.txt   9
6519_2021JAN_jobAD.txt   3
6521_2021JAN_jobAD.txt   6
6527_2021JAN_jobAD.txt   1
6530_2021JAN_jobAD.txt   12
6531_2021JAN_jobAD.txt   14
6533_2021JAN_jobAD.txt   48
6537_2021JAN_jobAD.txt   41
6542_2021JAN_jobAD.txt   5
6543_2021JAN_jobAD.txt   11
6545_2021JAN_jobAD.txt   15
6549_2021JAN_jobAD.txt   27
6550_2021JAN_jobAD.txt   39
6554_2021JAN_jobAD.txt   20
6556_2021JAN_jobAD.txt   3
6561_2021JAN_jobAD.txt   4
6562_2021JAN_jobAD.txt   28
6564_2021JAN_jobAD.txt   5
6566_2021JAN_jobAD.txt   14
6572_2021JAN_jobAD.txt   2
6573_2021JAN_jobAD.txt   4
6579_2021JAN_jobAD.txt   3
6581_2021JAN_jobAD.txt   11
6582_2021JAN_jobAD.txt   49
6583_2021JAN_jobAD.txt   1
6584_2021JAN_jobAD.txt   7
6585_2021JAN_jobAD.txt   11
6587_2021JAN_jobAD.txt   44
6588_2021JAN_jobAD.txt   17
6589_2021JAN_jobAD.txt   22
6592_202

In [17]:
ITJobsList = []
for i in range(0, len(urlList)):
    filename = "./dat/"+str(i+1)+"_2021JAN_jobAD.txt"
    if (os.path.exists(filename)):
        with open(filename, encoding='utf-8') as fin:
            all_ITs = re.findall(r"(?=("+'|'.join(itList)+r"))", fin.read())
            nITs = len(all_ITs)
            if nITs > 0:
                print('tech? ',str(i+1)+"_2021JAN_jobAD.txt",' ',nITs)
                ITJobsList.append(i+1)
            else:
                continue
    else:
        continue
        
print(ITJobsList)

tech?  1_2021JAN_jobAD.txt   7
tech?  22_2021JAN_jobAD.txt   1
tech?  28_2021JAN_jobAD.txt   3
tech?  34_2021JAN_jobAD.txt   1
tech?  41_2021JAN_jobAD.txt   1
tech?  54_2021JAN_jobAD.txt   7
tech?  55_2021JAN_jobAD.txt   2
tech?  65_2021JAN_jobAD.txt   1
tech?  74_2021JAN_jobAD.txt   4
tech?  92_2021JAN_jobAD.txt   1
tech?  96_2021JAN_jobAD.txt   1
tech?  118_2021JAN_jobAD.txt   1
tech?  121_2021JAN_jobAD.txt   6
tech?  124_2021JAN_jobAD.txt   2
tech?  131_2021JAN_jobAD.txt   1
tech?  153_2021JAN_jobAD.txt   1
tech?  168_2021JAN_jobAD.txt   3
tech?  169_2021JAN_jobAD.txt   4
tech?  176_2021JAN_jobAD.txt   1
tech?  177_2021JAN_jobAD.txt   4
tech?  192_2021JAN_jobAD.txt   5
tech?  195_2021JAN_jobAD.txt   1
tech?  206_2021JAN_jobAD.txt   3
tech?  213_2021JAN_jobAD.txt   3
tech?  240_2021JAN_jobAD.txt   3
tech?  247_2021JAN_jobAD.txt   4
tech?  253_2021JAN_jobAD.txt   4
tech?  266_2021JAN_jobAD.txt   1
tech?  268_2021JAN_jobAD.txt   19
tech?  277_2021JAN_jobAD.txt   1
tech?  310_2021JAN_jo

tech?  2541_2021JAN_jobAD.txt   6
tech?  2551_2021JAN_jobAD.txt   2
tech?  2554_2021JAN_jobAD.txt   1
tech?  2570_2021JAN_jobAD.txt   8
tech?  2580_2021JAN_jobAD.txt   2
tech?  2585_2021JAN_jobAD.txt   5
tech?  2597_2021JAN_jobAD.txt   6
tech?  2624_2021JAN_jobAD.txt   21
tech?  2639_2021JAN_jobAD.txt   7
tech?  2648_2021JAN_jobAD.txt   1
tech?  2652_2021JAN_jobAD.txt   2
tech?  2658_2021JAN_jobAD.txt   1
tech?  2665_2021JAN_jobAD.txt   11
tech?  2671_2021JAN_jobAD.txt   1
tech?  2683_2021JAN_jobAD.txt   4
tech?  2684_2021JAN_jobAD.txt   1
tech?  2687_2021JAN_jobAD.txt   16
tech?  2696_2021JAN_jobAD.txt   3
tech?  2707_2021JAN_jobAD.txt   2
tech?  2713_2021JAN_jobAD.txt   3
tech?  2714_2021JAN_jobAD.txt   1
tech?  2717_2021JAN_jobAD.txt   2
tech?  2720_2021JAN_jobAD.txt   1
tech?  2721_2021JAN_jobAD.txt   5
tech?  2725_2021JAN_jobAD.txt   2
tech?  2730_2021JAN_jobAD.txt   4
tech?  2736_2021JAN_jobAD.txt   1
tech?  2738_2021JAN_jobAD.txt   1
tech?  2739_2021JAN_jobAD.txt   5
tech?  2741

tech?  4183_2021JAN_jobAD.txt   1
tech?  4186_2021JAN_jobAD.txt   2
tech?  4187_2021JAN_jobAD.txt   4
tech?  4191_2021JAN_jobAD.txt   1
tech?  4195_2021JAN_jobAD.txt   3
tech?  4198_2021JAN_jobAD.txt   13
tech?  4202_2021JAN_jobAD.txt   3
tech?  4210_2021JAN_jobAD.txt   1
tech?  4215_2021JAN_jobAD.txt   2
tech?  4219_2021JAN_jobAD.txt   1
tech?  4221_2021JAN_jobAD.txt   8
tech?  4223_2021JAN_jobAD.txt   3
tech?  4225_2021JAN_jobAD.txt   1
tech?  4230_2021JAN_jobAD.txt   1
tech?  4237_2021JAN_jobAD.txt   2
tech?  4241_2021JAN_jobAD.txt   2
tech?  4242_2021JAN_jobAD.txt   8
tech?  4245_2021JAN_jobAD.txt   2
tech?  4253_2021JAN_jobAD.txt   3
tech?  4254_2021JAN_jobAD.txt   4
tech?  4255_2021JAN_jobAD.txt   1
tech?  4256_2021JAN_jobAD.txt   7
tech?  4267_2021JAN_jobAD.txt   3
tech?  4271_2021JAN_jobAD.txt   4
tech?  4274_2021JAN_jobAD.txt   50
tech?  4278_2021JAN_jobAD.txt   10
tech?  4289_2021JAN_jobAD.txt   1
tech?  4293_2021JAN_jobAD.txt   16
tech?  4295_2021JAN_jobAD.txt   1
tech?  431

tech?  5881_2021JAN_jobAD.txt   1
tech?  5885_2021JAN_jobAD.txt   8
tech?  5894_2021JAN_jobAD.txt   4
tech?  5897_2021JAN_jobAD.txt   4
tech?  5905_2021JAN_jobAD.txt   8
tech?  5908_2021JAN_jobAD.txt   1
tech?  5911_2021JAN_jobAD.txt   1
tech?  5912_2021JAN_jobAD.txt   3
tech?  5913_2021JAN_jobAD.txt   1
tech?  5918_2021JAN_jobAD.txt   8
tech?  5927_2021JAN_jobAD.txt   26
tech?  5928_2021JAN_jobAD.txt   10
tech?  5943_2021JAN_jobAD.txt   2
tech?  5945_2021JAN_jobAD.txt   2
tech?  5946_2021JAN_jobAD.txt   2
tech?  5956_2021JAN_jobAD.txt   4
tech?  5965_2021JAN_jobAD.txt   25
tech?  5983_2021JAN_jobAD.txt   2
tech?  5985_2021JAN_jobAD.txt   3
tech?  5987_2021JAN_jobAD.txt   2
tech?  5995_2021JAN_jobAD.txt   1
tech?  5996_2021JAN_jobAD.txt   1
tech?  6005_2021JAN_jobAD.txt   9
tech?  6009_2021JAN_jobAD.txt   8
tech?  6014_2021JAN_jobAD.txt   6
tech?  6019_2021JAN_jobAD.txt   1
tech?  6027_2021JAN_jobAD.txt   2
tech?  6035_2021JAN_jobAD.txt   8
tech?  6043_2021JAN_jobAD.txt   9
tech?  6064

In [18]:
USAITJobsList = []
fout = open("./ads/2021JAN_USAITjobAD.txt","w")
for i in range(0, len(ITJobsList)):
    filename = "./dat/"+str(ITJobsList[i])+"_2021JAN_jobAD.txt"
    if (os.path.exists(filename)):
        with open(filename, encoding='utf-8') as fin:
            all_USAs = re.findall(r"(?=("+'|'.join(statesList)+r"))", fin.read())
            nUSAs = len(all_USAs)
            if nUSAs > 0:
                print('USA tech? ',str(ITJobsList[i])+"_2021JAN_jobAD.txt",' ',nUSAs)
                fout.write(str(ITJobsList[i])+'_2021JAN_jobAD.txt'+' '+str(nUSAs)+'\n')
                USAITJobsList.append(ITJobsList[i])
            else:
                continue
    else:
        continue
            
fout.close()
print(USAITJobsList)

USA tech?  1_2021JAN_jobAD.txt   49
USA tech?  22_2021JAN_jobAD.txt   3
USA tech?  28_2021JAN_jobAD.txt   4
USA tech?  41_2021JAN_jobAD.txt   4
USA tech?  54_2021JAN_jobAD.txt   45
USA tech?  55_2021JAN_jobAD.txt   17
USA tech?  65_2021JAN_jobAD.txt   3
USA tech?  74_2021JAN_jobAD.txt   3
USA tech?  96_2021JAN_jobAD.txt   5
USA tech?  121_2021JAN_jobAD.txt   4
USA tech?  124_2021JAN_jobAD.txt   18
USA tech?  153_2021JAN_jobAD.txt   5
USA tech?  168_2021JAN_jobAD.txt   24
USA tech?  169_2021JAN_jobAD.txt   3
USA tech?  177_2021JAN_jobAD.txt   3
USA tech?  192_2021JAN_jobAD.txt   1
USA tech?  195_2021JAN_jobAD.txt   1
USA tech?  206_2021JAN_jobAD.txt   2
USA tech?  240_2021JAN_jobAD.txt   3
USA tech?  247_2021JAN_jobAD.txt   1
USA tech?  253_2021JAN_jobAD.txt   3
USA tech?  266_2021JAN_jobAD.txt   5
USA tech?  268_2021JAN_jobAD.txt   14
USA tech?  277_2021JAN_jobAD.txt   10
USA tech?  323_2021JAN_jobAD.txt   3
USA tech?  332_2021JAN_jobAD.txt   3
USA tech?  334_2021JAN_jobAD.txt   1
USA 

USA tech?  2551_2021JAN_jobAD.txt   184
USA tech?  2554_2021JAN_jobAD.txt   2
USA tech?  2570_2021JAN_jobAD.txt   3
USA tech?  2580_2021JAN_jobAD.txt   184
USA tech?  2585_2021JAN_jobAD.txt   10
USA tech?  2597_2021JAN_jobAD.txt   43
USA tech?  2624_2021JAN_jobAD.txt   55
USA tech?  2639_2021JAN_jobAD.txt   20
USA tech?  2648_2021JAN_jobAD.txt   2
USA tech?  2658_2021JAN_jobAD.txt   22
USA tech?  2665_2021JAN_jobAD.txt   11
USA tech?  2671_2021JAN_jobAD.txt   11
USA tech?  2683_2021JAN_jobAD.txt   11
USA tech?  2684_2021JAN_jobAD.txt   4
USA tech?  2687_2021JAN_jobAD.txt   18
USA tech?  2696_2021JAN_jobAD.txt   2
USA tech?  2707_2021JAN_jobAD.txt   5
USA tech?  2713_2021JAN_jobAD.txt   7
USA tech?  2714_2021JAN_jobAD.txt   2
USA tech?  2717_2021JAN_jobAD.txt   1
USA tech?  2720_2021JAN_jobAD.txt   76
USA tech?  2721_2021JAN_jobAD.txt   3
USA tech?  2730_2021JAN_jobAD.txt   20
USA tech?  2736_2021JAN_jobAD.txt   9
USA tech?  2738_2021JAN_jobAD.txt   4
USA tech?  2739_2021JAN_jobAD.txt  

USA tech?  4128_2021JAN_jobAD.txt   177
USA tech?  4159_2021JAN_jobAD.txt   3
USA tech?  4183_2021JAN_jobAD.txt   4
USA tech?  4186_2021JAN_jobAD.txt   4
USA tech?  4187_2021JAN_jobAD.txt   20
USA tech?  4195_2021JAN_jobAD.txt   31
USA tech?  4198_2021JAN_jobAD.txt   20
USA tech?  4202_2021JAN_jobAD.txt   4
USA tech?  4210_2021JAN_jobAD.txt   2
USA tech?  4215_2021JAN_jobAD.txt   8
USA tech?  4219_2021JAN_jobAD.txt   5
USA tech?  4221_2021JAN_jobAD.txt   3
USA tech?  4225_2021JAN_jobAD.txt   2
USA tech?  4230_2021JAN_jobAD.txt   4
USA tech?  4237_2021JAN_jobAD.txt   1
USA tech?  4241_2021JAN_jobAD.txt   187
USA tech?  4242_2021JAN_jobAD.txt   21
USA tech?  4245_2021JAN_jobAD.txt   180
USA tech?  4253_2021JAN_jobAD.txt   1
USA tech?  4254_2021JAN_jobAD.txt   7
USA tech?  4255_2021JAN_jobAD.txt   10
USA tech?  4256_2021JAN_jobAD.txt   5
USA tech?  4267_2021JAN_jobAD.txt   39
USA tech?  4271_2021JAN_jobAD.txt   11
USA tech?  4274_2021JAN_jobAD.txt   39
USA tech?  4278_2021JAN_jobAD.txt   

USA tech?  5612_2021JAN_jobAD.txt   2
USA tech?  5619_2021JAN_jobAD.txt   33
USA tech?  5624_2021JAN_jobAD.txt   17
USA tech?  5627_2021JAN_jobAD.txt   8
USA tech?  5631_2021JAN_jobAD.txt   8
USA tech?  5637_2021JAN_jobAD.txt   32
USA tech?  5642_2021JAN_jobAD.txt   2
USA tech?  5645_2021JAN_jobAD.txt   2
USA tech?  5646_2021JAN_jobAD.txt   154
USA tech?  5649_2021JAN_jobAD.txt   8
USA tech?  5651_2021JAN_jobAD.txt   20
USA tech?  5653_2021JAN_jobAD.txt   8
USA tech?  5662_2021JAN_jobAD.txt   2
USA tech?  5664_2021JAN_jobAD.txt   14
USA tech?  5676_2021JAN_jobAD.txt   3
USA tech?  5689_2021JAN_jobAD.txt   24
USA tech?  5693_2021JAN_jobAD.txt   9
USA tech?  5700_2021JAN_jobAD.txt   5
USA tech?  5710_2021JAN_jobAD.txt   13
USA tech?  5719_2021JAN_jobAD.txt   1
USA tech?  5746_2021JAN_jobAD.txt   1
USA tech?  5750_2021JAN_jobAD.txt   2
USA tech?  5766_2021JAN_jobAD.txt   2
USA tech?  5785_2021JAN_jobAD.txt   31
USA tech?  5795_2021JAN_jobAD.txt   3
USA tech?  5815_2021JAN_jobAD.txt   2
US

In [19]:
USAITQAsList = []
fout = open("./ads/2021JAN_USAITQAs.txt","w")
for i in range(0, len(USAITJobsList)):
    filename = "./dat/"+str(USAITJobsList[i])+"_2021JAN_jobAD.txt"
    if (os.path.exists(filename)):
        with open(filename, encoding='utf-8') as fin:
            all_QAs = re.findall(r"(?=("+'|'.join(qaList)+r"))", fin.read())
            nQAs = len(all_QAs)
            if nQAs > 0:
                print('USA tech QAs? ',str(USAITJobsList[i])+"_2021JAN_jobAD.txt",' ',nQAs)
                fout.write(str(USAITJobsList[i])+'_2021JAN_jobAD.txt'+' '+str(nQAs)+'\n')
                USAITQAsList.append(USAITJobsList[i])
            else:
                continue
    else:
        continue
            
fout.close()
print(USAITQAsList)

USA tech QAs?  28_2021JAN_jobAD.txt   2
USA tech QAs?  96_2021JAN_jobAD.txt   2
USA tech QAs?  575_2021JAN_jobAD.txt   1
USA tech QAs?  578_2021JAN_jobAD.txt   3
USA tech QAs?  610_2021JAN_jobAD.txt   3
USA tech QAs?  625_2021JAN_jobAD.txt   11
USA tech QAs?  681_2021JAN_jobAD.txt   3
USA tech QAs?  683_2021JAN_jobAD.txt   1
USA tech QAs?  763_2021JAN_jobAD.txt   1
USA tech QAs?  786_2021JAN_jobAD.txt   1
USA tech QAs?  807_2021JAN_jobAD.txt   1
USA tech QAs?  809_2021JAN_jobAD.txt   3
USA tech QAs?  810_2021JAN_jobAD.txt   8
USA tech QAs?  833_2021JAN_jobAD.txt   1
USA tech QAs?  836_2021JAN_jobAD.txt   2
USA tech QAs?  864_2021JAN_jobAD.txt   2
USA tech QAs?  909_2021JAN_jobAD.txt   1
USA tech QAs?  952_2021JAN_jobAD.txt   2
USA tech QAs?  962_2021JAN_jobAD.txt   4
USA tech QAs?  971_2021JAN_jobAD.txt   1
USA tech QAs?  992_2021JAN_jobAD.txt   1
USA tech QAs?  993_2021JAN_jobAD.txt   1
USA tech QAs?  1018_2021JAN_jobAD.txt   1
USA tech QAs?  1019_2021JAN_jobAD.txt   1
USA tech QAs?  